In [ ]:
> **Process outline:**
>
>> **STEP 1:**
>> Obtain a pre-trained CNN Model
>> 
>> **STEP 2:** Remove the last few layers
>> 
>> **STEP 3:** Add layers to the end of the pre-trained CNN Model to classify our data
>> 
>> **STEP 4:** Hold the other layers static
>> 
>> **STEP 5:**  train the new model
>> 

**STEP 1: Data Generator**

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image dimensions and data generator with a validation split
height = 600
width = 800
channels = 3

train_datagen = ImageDataGenerator(
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Setting 20% of the images for validation
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    '/Users/raghadibrahim/Desktop/Olive-Tree-Disease-Detection/dataset/training',
    target_size=(height, width),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',
    subset='training',
    seed=123
)

# Validation generator
validation_generator = train_datagen.flow_from_directory(
    '/Users/raghadibrahim/Desktop/Olive-Tree-Disease-Detection/dataset/training',
    target_size=(height, width),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',
    subset='validation',  
    seed=123
)

# Testing generator
test_generator = test_datagen.flow_from_directory(
    '/Users/raghadibrahim/Desktop/Olive-Tree-Disease-Detection/dataset/testing',
    target_size=(height, width),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',  
    seed=123
)

---

**STEP 2: Setting up a Pre-Trained CNN ResNet-50 Model**

from tensorflow.keras.applications import ResNet50V2

res_model = ResNet50V2(weights='imagenet',
                   include_top=False,
                   input_shape=(600,800,3))
res_model.summary()

**STEP 3: Freeze layers**

# Freeze all the layers in the base model
for layer in res_model.layers:
    layer.trainable = False

#### ResNet-50

import random

def set_random_seed(seed):
    
    '''Random seeds for reproducability'''
    
    random.seed(seed)
    tf.random.set_seed(seed)

from tensorflow.keras.models import Model
set_random_seed(123)
x = res_model.output

x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
output = Dense(3, activation='softmax')(x)

model = Model(inputs=res_model.input, 
              outputs = output)

**STEP 4: Train the model**

from tensorflow.keras.callbacks import EarlyStopping

set_random_seed(121)
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Use an early stopping callback to stop training
# once we no longer have improvements in our validation loss
early_stop = EarlyStopping(monitor='val_loss', 
                           patience=4, 
                           mode='min', 
                           verbose=1)
model.fit(train_generator,
          epochs=10,
          validation_data = validation_generator)

###  Evaluate the Model

# Evaluate the model's performance on the test data

score = model.evaluate(test_generator, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

